## 1. Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import threading
import numpy as np

## 2. Function

### a. Get all of title from url

In [2]:
def get_title_from_url(url, num):
    ser = Service("D:/chromedriver.exe")
    driver = webdriver.Chrome(service = ser)
    driver.get(url)
    driver.fullscreen_window()
    time.sleep(2)
    titles = set()
    while True:
        try:
            ## Click button "See more"
            button = driver.find_element(By.CSS_SELECTOR, 'button.text.control__loadmore')
            button.click()
            time.sleep(1)
            ## Collect title
            soup = BeautifulSoup(driver.page_source, "html.parser")
            articles = soup.find_all("article", {"class" : "story story-small"})
            for article in articles:
                title = article.find("a", {"class" : "cms-link"}).get("title")
                titles.add(title.rstrip())
            if len(titles) > num:
                break
        except NoSuchElementException:
            break
    return titles

### b. Save to file

In [3]:
def save_to_file(type, titles):
    path = str(f"./raw_data/{type}.txt")
    with open(path, "w", encoding="utf-8") as f:
        for title in titles:
            f.write(title + "\n")

### c. Thread

In [5]:
def get_and_save_title(num, type):
    url = f"https://nhandan.vn/{type}/"
    titles = get_title_from_url(url, num)
    save_to_file(type, titles)

## 3. Execute

In [ ]:
types = ["vanhoa", "phapluat", "thethao", "giaoduc", "chinhtri", "kinhte", "xahoi"]
num = 2000
threads = []
for type in types:
    thread = threading.Thread(target=get_and_save_title, args=(num, type))
    thread.start()
    threads.append(thread)

for thread in threads:
    thread.join()